# Self-Driving Car Engineer Nanodegree


## Project 4: **Advanced Lane Finding** 

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

The images for camera calibration are stored in the folder called camera_cal. The images in test_images are for testing your pipeline on single frames. If you want to extract more test images from the videos, you can simply use an image writing method like cv2.imwrite(), i.e., you can read the video in frame by frame as usual, and for frames you want to save for later you can write to an image file.

To help the reviewer examine your work, please save examples of the output from each stage of your pipeline in the folder called output_images, and include a description in your writeup for the project of what each image shows. The video called project_video.mp4 is the video your pipeline should work well on.

The challenge_video.mp4 video is an extra (and optional) challenge for you if you want to test your pipeline under somewhat trickier conditions. The harder_challenge.mp4 video is another optional challenge and is brutal!

If you're feeling ambitious (again, totally optional though), don't stop there! We encourage you to go out and take video of your own, calibrate your camera and show us how you would implement this project from scratch!

## Import Libraries

In [1]:
# Import necessary packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import numpy as np
import cv2
import os
import random
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

print("#########################################")
print("Packages imported!")
print("#########################################")

#########################################
Packages imported!
#########################################


## Import camera calibration data as global variables

In [2]:
# Load pickle file which holds calculated camera parameters
with open('./camera_cal_parameters.pkl','rb') as f:
    [mtx,dist] = pickle.load(f)
    # Define parameters as global variables
    global mtx
    global dist
print("#########################################")
print('Camera calibration data successfully loaded from pickle file.')
print('Variable "mtx":')
print(mtx)
print('Variable "dist":')
print(dist)
print("#########################################")

#########################################
Camera calibration data successfully loaded from pickle file.
Variable "mtx":
[[1.15396093e+03 0.00000000e+00 6.69705357e+02]
 [0.00000000e+00 1.14802496e+03 3.85656234e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Variable "dist":
[[-2.41017956e-01 -5.30721173e-02 -1.15810355e-03 -1.28318856e-04
   2.67125290e-02]]
#########################################


## Helper functions

In [3]:
def readimage(filename):
    # Read image file using mpimg (ensuring RGB color space)
    return mpimg.imread(filename)


def undistort_img(img,mtx,dist):
    # Undistort image file using given camera coefficients mtx and dist
    return(cv2.undistort(img, mtx, dist, None, mtx))


def grayscale_conv(img_rgb):
    # Applies the Grayscale transform
    return cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)


def hls_conv(img_rgb):
    # Applies RGB to HLS color space transformation
    return cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HLS)


def thresholding(img_rgb):
    # Define threshold values for image thresholding of HLS and GRAYSCALE image: 
    # only accept low Hue values and high staturation values. Saturation is ignored
    HUE = [0,120]  # 0,80
    LIGHTNESS = [0,255]
    SATURATION = [120,255]  # 150,255
    GRAY = [100,255]
    SOBELX = [30,255]
    
    # First: process HLS color space information
    # Convert image to HLS color space
    img_hls = hls_conv(img_rgb)
    img_h = img_hls[:,:,0]
    img_l = img_hls[:,:,1]
    img_s = img_hls[:,:,2]
    # Transform threshold values to necessary data type
    lim_lo = np.array([HUE[0], LIGHTNESS[0], SATURATION[0]], dtype=np.uint8)
    lim_hi = np.array([HUE[1], LIGHTNESS[1], SATURATION[1]], dtype=np.uint8)
    # Apply threshold to HLS image
    mask_h = np.zeros_like(img_h)
    mask_h[(img_h >= HUE[0])        & (img_h <= HUE[1])]         = 1
    mask_l = np.zeros_like(img_l)
    mask_l[(img_l >= LIGHTNESS[0])  & (img_l <= LIGHTNESS[1])]   = 1
    mask_s = np.zeros_like(img_s)
    mask_s[(img_s >= SATURATION[0]) & (img_s <= SATURATION[1])]  = 1
    mask_hls = mask_h & mask_l & mask_s
    
    # SECOND: process grayscale information
    img_gray = grayscale_conv(img_rgb)    
    # Detect white areas (on GRAYSCALE image)
    lim_lo = np.array(GRAY[0], dtype=np.uint8)
    lim_hi = np.array(GRAY[1], dtype=np.uint8)
    # Apply threshold to Grayscale image
    mask_gray = np.zeros_like(img_h)
    mask_gray[(img_gray >= GRAY[0]) & (img_gray <= GRAY[1])]     = 1
    
    # Apply masks to images
    img_masked_gray = cv2.bitwise_and(img_gray, img_gray, mask=mask_gray)
    # Apply sobel x gradient detection on masked grayscale image
    sobelx = cv2.Sobel(img_masked_gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    # Threshold sobel x gradient
    mask_sx = np.zeros_like(scaled_sobel)
    mask_sx[(scaled_sobel >= SOBELX[0]) & (scaled_sobel <= SOBELX[1])] = 1

    # THIRD: Stack all channels
    # Create combined B/W image
    thresholded_bw = np.zeros_like(mask_hls)
    thresholded_bw[(mask_hls == 1) | (mask_sx == 1)] = 1    
    # Create combined and colored mask image
    thresholded_color = np.dstack((mask_hls, mask_sx, np.zeros_like(mask_sx))) * 255

    # Return all information
    return (thresholded_bw,thresholded_color)    


def perspective_transform(img,direction=1):
    # Function to perform perspective transformation. Transformation is possible in both directions
    # Direction has to be set accordingly:
    #     "direction = 1" : regular image to bird's eye view (forward transformation)
    #     "direction = 2" : bird's eye view to regular image (backward transformation)
    
    # Define points to use for source / destination
    src = np.array([[188,675],[586,450],[694,450],[1092,675]], np.float32)   # new symmetric points
    dst = np.array([[280,720],[280,0],[1000,0],[1000,720]], np.float32)      # new symmetric points

    # Determine matrix to perform perspective transformation
    if direction == 1:
        M_transform = cv2.getPerspectiveTransform(src,dst)
    elif direction == 2:
        M_transform = cv2.getPerspectiveTransform(dst,src)
    
    # Perform perspective transformation and return warped image
    return cv2.warpPerspective(img,M_transform,(img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)

print("#########################################")
print("Helper functions loaded!")
print("#########################################")

#########################################
Helper functions loaded!
#########################################


## Lane detection helper functions

In [4]:
def lane_detection_initial(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    try:
        left_fit = np.polyfit(lefty, leftx, 2)
        left_fit_m = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    except:
        left_fit = [0,0,0]
        left_fit_m = [0,0,0]
    try:
        right_fit = np.polyfit(righty, rightx, 2)
        right_fit_m = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    except:
        right_fit = [0,0,0]
        right_fit_m = [0,0,0]
    
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
      
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    pts_left = np.column_stack((np.int32(left_fitx),np.int32(ploty)))
    pts_right = np.column_stack((np.int32(right_fitx),np.int32(ploty)))

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    cv2.polylines(out_img, [pts_left], isClosed=False, color=(255,255,0), thickness=3)
    cv2.polylines(out_img, [pts_right], isClosed=False, color=(255,255,0), thickness=3)
    
    return (out_img, left_fit, right_fit, left_fit_m, right_fit_m)
    

    
def lane_detection_consec(binary_warped,left_fit,right_fit):
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Fit a second order polynomial to each
    try:
        left_fit = np.polyfit(lefty, leftx, 2)
        left_fit_m = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    except:
        left_fit = [0,0,0]
        left_fit_m = [0,0,0]    
    try:
        right_fit = np.polyfit(righty, rightx, 2)
        right_fit_m = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    except:
        right_fit = [0,0,0]
        right_fit_m = [0,0,0]
    # Fit new polynomials to x,y in world space
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    ######################################################
        
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    out_img2 = np.dstack((binary_warped, binary_warped, binary_warped))*255
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Create an image to draw on and an image to show the selection window
    window_img = np.zeros_like(out_img2)
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))

    pts_left = np.column_stack((np.int32(left_fitx),np.int32(ploty)))
    pts_right = np.column_stack((np.int32(right_fitx),np.int32(ploty)))

    result = cv2.addWeighted(out_img2, 1, window_img, 0.3, 0)
    cv2.polylines(result, [pts_left], isClosed=False, color=(255,255,0), thickness=3)
    cv2.polylines(result, [pts_right], isClosed=False, color=(255,255,0), thickness=3)
    
        
    return (result, left_fit, right_fit, left_fit_m, right_fit_m)
   
    
    
# bw_warped = perspective_transform(bw_binary)[0]    
# left_fit, right_fit, left_fit_m, right_fit_m, out_img = lane_detection(bw_warped)

## Curve radii and vehicle position on lane helper functions

In [5]:
def curve_radius(left_fit_m, right_fit_m, img_shape):
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = img_shape[0]-1
    
    # Conversion factors for pixels to meters are read from global variables
#     ym_per_pix = 30/720 # meters per pixel in y dimension
#     xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Calculate curve radius in METRIC space
    left_curverad_m = \
    ((1 + (2*left_fit_m[0]*y_eval*ym_per_pix + left_fit_m[1])**2)**1.5) / np.absolute(2*left_fit_m[0])
    right_curverad_m = \
    ((1 + (2*right_fit_m[0]*y_eval*ym_per_pix + right_fit_m[1])**2)**1.5) / np.absolute(2*right_fit_m[0])
    return 0.5*(left_curverad_m + right_curverad_m)
    
def veh_position(left_fit, right_fit, img_shape):
    # Calculate vehicle position
    y_val = img_shape[0]-1
    left_x = left_fit[0]*y_val**2 + left_fit[1]*y_val + left_fit[2]
    right_x = right_fit[0]*y_val**2 + right_fit[1]*y_val + right_fit[2]
    
    x_veh_ctr = (img_shape[1]-1)*0.5
    
    x_lane_pos = left_x + 0.5*(right_x - left_x)
    
    x_dist = x_veh_ctr - x_lane_pos
    
    x_dist_m = x_dist * xm_per_pix
    return(x_dist_m)

## Prepare output image with overlayed information

In [6]:
def birdseye_lane_info(left_fit,right_fit,img):
    # Prepare empty image to draw onto
    out_img = np.zeros_like(img)
    
    # Generate x and y values for plotting
    margin = 15
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]    # Generate a polygon to show lane line
    
    # Prepare polygon information for cv2 fillpoly function
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin,ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin,ploty])))])    
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    lane_window1 = np.array([np.transpose(np.vstack([left_fitx+margin, ploty]))])
    lane_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx-margin, ploty])))])    
    lane_window_pts = np.hstack((lane_window1,lane_window2))
    
    # Plot polygons onto empty output image
    cv2.fillPoly(out_img, np.int_([left_line_pts]), (255, 0, 0))
    cv2.fillPoly(out_img, np.int_([right_line_pts]), (0, 0, 255))
    cv2.fillPoly(out_img, np.int_([lane_window_pts]), (0,255 , 0))

    # Return image file with overlay
    return (out_img)


# def plot_additional_info(img,img_bw,img_lanes):
def plot_additional_info(img,sub_img1,sub_img2):
    img_add = np.zeros_like(img)
    vid_add1 = np.zeros_like(img)
    vid_add2 = np.zeros_like(img)
    out_img = np.copy(img)
    
    # Define points for drawing frame, segmentation images and text
    pts_frame = np.array([[10,10],[10,240],[np.shape(img)[1]-11,240],[np.shape(img)[1]-11,10]], np.int32)
    pts_img1 = np.array([[15,15],[15,235],[406,235],[406,15]], np.int32)
    pts_img2 = np.array([[416,15],[416,235],[807,235],[807,15]], np.int32)
    
    # Fill areas
    cv2.fillPoly(img_add, [pts_frame], (156,158,148))
    cv2.fillPoly(vid_add1, [pts_img1], (1,1,1))
    cv2.fillPoly(vid_add2, [pts_img2], (1,1,1))
    
    # Prepare mini video overlay
    sub_img1s = cv2.resize(sub_img1, (391, 220)) 
    sub_img2s = cv2.resize(sub_img2, (391, 220)) 

    
    # Draw grayish frame on image
    inds = img_add.nonzero();
    overlay = cv2.addWeighted(out_img, 0.3, img_add, 0.9, 0)    
    out_img[inds] = overlay[inds]
    
    # Draw segmentation images / videos on image
    inds = vid_add1.nonzero();
    out_img[pts_img1[0,1]:pts_img1[2,1],pts_img1[0,0]:pts_img1[2,0]] = sub_img1s
    out_img[pts_img2[0,1]:pts_img2[2,1],pts_img2[0,0]:pts_img2[2,0]] = sub_img2s
    
    # Overlay text
    fontface = cv2.FONT_HERSHEY_DUPLEX
    fontface2 = cv2.FONT_HERSHEY_COMPLEX_SMALL
    fontcolor = (0,0,0)
    fontscale = 0.5
    linetype = cv2.LINE_AA
    radius = np.mean(n_curve_rad[np.nonzero(n_curve_rad)]) 
    dist = np.mean(n_dist_dev[np.nonzero(n_dist_dev)]) 
    cv2.putText(out_img,'Position vs. lane center: ',(820,40),1,fontface,fontcolor,2,linetype)
    cv2.putText(out_img,"%+1.1F meter" %(dist),(820,80),1,fontface,fontcolor,1,linetype)
    cv2.putText(out_img,'Curve radius: ',(820,180),1,fontface,fontcolor,2,linetype)
    if radius < 3500:
        cv2.putText(out_img,"%4d meter" %(radius),(820,220),1,fontface,fontcolor,1,linetype)
    else:
        cv2.putText(out_img,"straight lane",(820,220),1,fontface,fontcolor,1,linetype)
            
    # Return final image 
    return out_img



## Pipeline

In [7]:
def img_pipeline(img,saveimgs=False):
    # Load global variables as writable
    global n, n_left_fits, n_right_fits, n_left_fit_curr, n_right_fit_curr, n_curve_rad, n_dist_dev

    # Undistortion image with camera coefficients
    img_undist = undistort_img(img,mtx,dist)

    # Threshold image to obtain binary image file
    img_thresh_bw, img_thresh_col = thresholding(img_undist)

    # Perspective transformation: birds-eye view
    img_bw_warped = perspective_transform(img_thresh_bw,1)

    # Decide on amount of analyzed frames whether to use simple or extensive lane line detection
    if ((np.count_nonzero(n_left_fits) < n_frames*3) | (np.count_nonzero(n_right_fits) < n_frames*3)):
        # More extensive lane detection routine
        img_lane_detect_warped, n_left_fit_curr, n_right_fit_curr, left_fit_m, right_fit_m = \
            lane_detection_initial(img_bw_warped)
        # Write obtained information to global variables
        n_left_fits[n] = n_left_fit_curr
        n_right_fits[n] = n_right_fit_curr
    else: 
        # Lane detection routine using stabilized lane information from previous frames
        img_lane_detect_warped, n_left_fits[n], n_right_fits[n], left_fit_m, right_fit_m = \
            lane_detection_consec(img_bw_warped, n_left_fit_curr, n_right_fit_curr)
        n_left_fit_curr = np.mean(n_left_fits,0)
        n_right_fit_curr = np.mean(n_right_fits,0)

    # Calculate curve radius
    n_curve_rad[n] = curve_radius(left_fit_m, right_fit_m, np.shape(img_bw_warped))

    # Calculate distance to lane center
    n_dist_dev[n] = veh_position(n_left_fit_curr, n_right_fit_curr, np.shape(img_bw_warped))
    
    # Generate clean lane information in birds-eye view with highlighted lane and lane lines
    img_lane_highlighted_warped = birdseye_lane_info(n_left_fit_curr,n_right_fit_curr,img)

    # Warp lane information back to regular view and generate combined image
    img_lane_highlighted = perspective_transform(img_lane_highlighted_warped,direction=2)
    img_highlighted = cv2.addWeighted(img_undist, 1, img_lane_highlighted, 0.6, 0)   
    
    # Plot additional info to video image: in between images, curve radius and distance to lane center
    img_final = plot_additional_info(img_highlighted,img_thresh_col,img_lane_detect_warped)
    
    if saveimgs == True:
        cv2.imwrite("./output_images/steps/01-img_original.jpg",cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        cv2.imwrite("./output_images/steps/02-img_undist.jpg",cv2.cvtColor(img_undist, cv2.COLOR_RGB2BGR))
        cv2.imwrite("./output_images/steps/03-img_thresh_bw.jpg",img_thresh_bw*255)
        cv2.imwrite("./output_images/steps/04-img_thresh_col.jpg",cv2.cvtColor(img_thresh_col, cv2.COLOR_RGB2BGR))
        cv2.imwrite("./output_images/steps/05-img_bw_warped.jpg",img_bw_warped*255)
        cv2.imwrite("./output_images/steps/06-img_lane_detect_warped.jpg",cv2.cvtColor(img_lane_detect_warped, cv2.COLOR_RGB2BGR))
        cv2.imwrite("./output_images/steps/07-img_lane_highlighted_warped.jpg",cv2.cvtColor(img_lane_highlighted_warped, cv2.COLOR_RGB2BGR))
        cv2.imwrite("./output_images/steps/08-img_lane_highlighted.jpg",cv2.cvtColor(img_lane_highlighted, cv2.COLOR_RGB2BGR))
        cv2.imwrite("./output_images/steps/09-img_highlighted.jpg",cv2.cvtColor(img_highlighted, cv2.COLOR_RGB2BGR))
        cv2.imwrite("./output_images/steps/10-img_final.jpg",cv2.cvtColor(img_final, cv2.COLOR_RGB2BGR))
    
    # Increase counter
    n = n + 1
    if n >= n_frames:
        # Reset counter to start again from 0 when n_frames is reached
        # This is effectively overwriting oldest values in global arrays
        n = 0

    return (img_final)
    


## Test run

In [8]:
if 2 > 10: 
    init_global_vars()
    img = readimage('./test_images/straight_lines2.jpg')
    img_out = img_pipeline(img,saveimgs=True)
    fig = plt.figure()
    plt.imshow(img_out);
    fig.set_figwidth(20)    
    fig.set_figheight(15)

## Process video files

In [9]:
# Define filenames to input video files
f_clip  = ['./project_video.mp4',
           './challenge_video.mp4',
           './harder_challenge_video.mp4']
# Define filenames for output video files
f_clip_out =   ['./output_videos/out1_project_video.mp4',
                './output_videos/out2_challenge_video.mp4',
                './output_videos/out3_harder_challenge_video.mp4']

def init_global_vars():
    # Prepare global variables
    n_frames = 10     # amount of frames to use for moving averages
    n = 0             # initialize counter variable
    n_left_fits = np.zeros((n_frames,3))   # pre-allocate array for left lane polyfit results
    n_right_fits = np.zeros((n_frames,3))  # pre-allocate array for right lane polyfit results
    n_left_fit_curr = np.zeros((1,3))   # pre-allocate array for left lane polyfit results
    n_right_fit_curr = np.zeros((1,3))  # pre-allocate array for right lane polyfit results
    n_curve_rad = np.zeros((n_frames,1))   
    n_dist_dev = np.zeros((n_frames,1))
    # Conversion factors from pixel to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    global n, n_frames, n_left_fits, n_right_fits, n_left_fit_curr, n_right_fit_curr
    global n_curve_rad, n_dist_dev, ym_per_pix, xm_per_pix   # declare variables as global

In [10]:
# Process first video clip
init_global_vars()
vidno = 1
# clip = VideoFileClip(f_clip).subclip(0,3)
clip = VideoFileClip(f_clip[vidno-1])
clip_out = clip.fl_image(img_pipeline)
%time clip_out.write_videofile(f_clip_out[vidno-1], audio=False)

# Process second video clip
init_global_vars()
vidno = 2
# clip1 = VideoFileClip(f_clip).subclip(0,3)
clip = VideoFileClip(f_clip[vidno-1])
clip_out = clip.fl_image(img_pipeline)
%time clip_out.write_videofile(f_clip_out[vidno-1], audio=False)

# Process second video clip
init_global_vars()
vidno = 3
# clip1 = VideoFileClip(f_clip).subclip(0,3)
clip = VideoFileClip(f_clip[vidno-1])
clip_out = clip.fl_image(img_pipeline)
%time clip_out.write_videofile(f_clip_out[vidno-1], audio=False)

ping
[MoviePy] >>>> Building video ./output_videos/out1_project_video.mp4
[MoviePy] Writing video ./output_videos/out1_project_video.mp4


  0%|          | 0/1261 [00:00<?, ?it/s]

ping


  0%|          | 2/1261 [00:00<06:07,  3.43it/s]

ping


...

100%|█████████▉| 1197/1200 [05:53<00:00,  3.39it/s]

pong
pong


100%|█████████▉| 1199/1200 [05:53<00:00,  3.39it/s]

pong


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/out3_harder_challenge_video.mp4 

CPU times: user 5min 29s, sys: 57.8 s, total: 6min 27s
Wall time: 5min 56s


In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(f_clip_out[0]))

In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(f_clip_out[1]))

In [13]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(f_clip_out[2]))